In [24]:
!pip install yfinance transformers torch vaderSentiment pandas numpy scikit-learn xgboost lightgbm

!pip install sentencepiece 
!pip install --upgrade pandas
!pip install --upgrade lightgbm
!pip install --upgrade dask



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
import torch
import yfinance as yf
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor




In [26]:

# Load TinyLlama model and tokenizer once (to avoid reloading)
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tinyllama_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")


Some parameters are on the meta device because they were offloaded to the disk.


# Save TinyLama Pickle file

In [27]:
import requests
from bs4 import BeautifulSoup
import lightgbm as lgb
from sklearn.model_selection import train_test_split


In [28]:
# from google.colab import drive

# # Mount Drive
# drive.mount('/content/drive')

**Fetch Stock Info**

In [29]:
# Load csv File
# df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/stock_data.csv")
df = pd.read_csv("stock_data.csv")

# Rename columns using a dictionary
df.rename(columns={"Ticker Symbol": "Ticker", "Volatility": "Volatility",
                  "Net Cash Flow" : "Market Cap", "GICS Sector": "Sector",
                  "P/E Ratio": "P/E Ratio", "Sharpe_Ratio": "Sharpe Ratio",
                  "Dividend_Yield (%)": "Dividend_Yield"}, inplace=True)

# Select 5 Random Rows
random_stocks = df.sample(n=4, random_state=42)  # Set random_state for reproducibility
# random_stocks


random_tickers = random_stocks['Ticker'].tolist()


In [30]:

# Define Stock Universe
stock_list = random_tickers

# Fetch Stock Data
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "ticker": ticker,
        "sector": info.get("sector", "Unknown"),
        "market cap": info.get("marketCap", 0),
        "volatility": info.get("beta", 1),
        "p/e ratio": info.get("trailingPE", None),
        "stock_price": info.get("currentPrice", None),
        "sharpeRatio": info.get("sharpeRatio", 0.5),
        "dividend_yield":info.get("dividendYield", 0)
    }

# Function to fetch recent news headlines for a stock
# def get_stock_news(ticker):
#     url = f"https://finance.yahoo.com/quote/{ticker}/news"
#     headers = {"User-Agent": "Mozilla/5.0"}
#     response = requests.get(url, headers=headers)
#     soup = BeautifulSoup(response.text, "html.parser")

#     headlines = []
#     for item in soup.find_all("h3", class_="Mb(5px)"):
#         headline = item.get_text()
#         headlines.append(headline)

#     return headlines[:5]  # Limit to top 5 headlines


def get_stock_news(ticker):
    stock = yf.Ticker(ticker)
    news = stock.news  # Fetch news data

    print("News Response:", news)  # Debugging print

    if not news:  # Check if news is empty
        return ["No news available for this stock"]

    headlines = [article.get("title", "No title found") for article in news[:2]]
    return headlines

In [31]:
print(get_stock_news("AAPL"))

News Response: [{'id': '48775d66-72a4-349b-9288-8677672dcfb1', 'content': {'id': '48775d66-72a4-349b-9288-8677672dcfb1', 'contentType': 'STORY', 'title': 'Apple Inc. (AAPL) AI Woes Grow as Siri Upgrades Pushed Back, Leadership Team Reshuffled', 'description': '', 'summary': 'We recently published a list of GTC Recap and Beyond: 10 AI Stocks on Investors’ Radar. In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against other AI stocks that are on investors’ radar. Jensen Huang’s GTC keynote may not have unpacked significant surprises, but his insights and […]', 'pubDate': '2025-03-26T01:56:12Z', 'displayTime': '2025-03-26T01:56:12Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/insidermonkey.com/bd47dc2e86b4a3086f845d200d9daf1f', 'originalWidth': 1456, 'originalHeight': 816, 'caption': '', 'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/fTPNobtT0bdcSxj2xXAamw--~B/aD04MT

In [32]:

# Function to analyze sentiment using Llama LLM
def analyze_sentiment_llm(headlines):
    responses = []
    
    for headline in headlines:
        prompt = f"Analyze the sentiment (Positive, Negative, Neutral) of this news: '{headline}'."
        
        inputs = tokenizer(prompt, return_tensors="pt")
        output = tinyllama_model.generate(**inputs, max_length=30)
        
        print("Raw Model Output:", output)  # Debugging step
        response_text = tokenizer.decode(output[0], skip_special_tokens=True)
        
        responses.append(response_text.strip())

    return responses


**Stock Sentiment Analysis**

In [33]:


# Function to analyze sentiment using Llama LLM
# def analyze_sentiment_llm(headlines):
#     responses = []
#     for headline in headlines:
#         prompt = f"Analyze the sentiment (Positive, Negative, Neutral) of this news: '{headline}'."
#         response = tinyllama_model(prompt)
#         responses.append(response["choices"][0]["text"].strip())

#     return responses

# Function to analyze stock sentiment
def analyze_stock_sentiment(ticker):
    # Fetch stock-related news headlines
    headlines = get_stock_news(ticker) or []  # Ensure it's not None

    if not headlines:  # If no headlines are available, return default sentiment scores
        return {"Stock": ticker, "POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}

    # Analyze sentiment using LLM
    llm_sentiments = analyze_sentiment_llm(headlines) or []  # Ensure it's a list

    # Initialize sentiment score counters
    sentiment_scores = {"POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}

    # Count occurrences of each sentiment
    for sentiment in llm_sentiments:
        sentiment = sentiment.lower()  # Convert to lowercase for consistency

        if "positive" in sentiment:
            sentiment_scores["POSITIVE"] += 1
        elif "negative" in sentiment:
            sentiment_scores["NEGATIVE"] += 1
        else:
            sentiment_scores["NEUTRAL"] += 1

    return {"Stock": ticker, **sentiment_scores}




In [34]:
analyze_stock_sentiment("SWN")

News Response: [{'id': '341a16d3-7adc-3747-aba8-8d3337bca9a0', 'content': {'id': '341a16d3-7adc-3747-aba8-8d3337bca9a0', 'contentType': 'STORY', 'title': 'Is Southwestern Energy Company (SWN) the Best Stock Under $15 To Buy Now?', 'description': '', 'summary': 'We recently compiled a list of 10 Best Stocks Under $15 To Buy Now. In this article, we will look at where Southwestern Energy Company (NYSE:SWN) ranks among the best stocks under $15. While September is usually the toughest month for the stock market, it is delivering solid results this time. Stocks are soaring after […]', 'pubDate': '2024-10-01T14:49:26Z', 'displayTime': '2024-10-01T14:49:26Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/insidermonkey.com/ebee65f2176b900b17d44579449d2d0e', 'originalWidth': 1456, 'originalHeight': 816, 'caption': '', 'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/2AbX4uePnXuan5cpNEgzMQ--~B/aD04MTY7dz0xNDU2O2Fwc

/Users/vijayalakshmig/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:2208: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])
Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])


{'Stock': 'SWN', 'POSITIVE': 2, 'NEGATIVE': 0, 'NEUTRAL': 0}

In [35]:
headlines = get_stock_news("SWN") or []
print("Fetched headlines:", headlines)  # Debugging print


News Response: [{'id': '341a16d3-7adc-3747-aba8-8d3337bca9a0', 'content': {'id': '341a16d3-7adc-3747-aba8-8d3337bca9a0', 'contentType': 'STORY', 'title': 'Is Southwestern Energy Company (SWN) the Best Stock Under $15 To Buy Now?', 'description': '', 'summary': 'We recently compiled a list of 10 Best Stocks Under $15 To Buy Now. In this article, we will look at where Southwestern Energy Company (NYSE:SWN) ranks among the best stocks under $15. While September is usually the toughest month for the stock market, it is delivering solid results this time. Stocks are soaring after […]', 'pubDate': '2024-10-01T14:49:26Z', 'displayTime': '2024-10-01T14:49:26Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/insidermonkey.com/ebee65f2176b900b17d44579449d2d0e', 'originalWidth': 1456, 'originalHeight': 816, 'caption': '', 'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/2AbX4uePnXuan5cpNEgzMQ--~B/aD04MTY7dz0xNDU2O2Fwc

**Collect stock data and prepare dataset**

In [36]:


# Function to collect stock data and prepare dataset
def collect_stock_data(stock_list):
    stock_data = []

    for ticker in stock_list:
        sentiment = analyze_stock_sentiment(ticker)
        fundamentals = get_stock_data(ticker)

        data = {**sentiment, **fundamentals}
        stock_data.append(data)

    df = pd.DataFrame(stock_data)
    df["Sentiment Score"] = df["POSITIVE"] - df["NEGATIVE"]
    df.dropna(inplace=True)  # Remove NaN values

    return df


In [37]:
# collect_stock_data(['SWN', 'ETN', 'ES', 'BBT', 'FBHS', 'NTRS', 'KO', 'SPGI', 'ZBH', 'AMT'])

**LGBM Model**

In [38]:

# Train LightGBM for Stock Ranking
def train_stock_ranking_model(df):
    X = df[["Sentiment Score", "p/e ratio", "market cap", "volatility"]]
    y = df["Sentiment Score"]  # Target: Higher sentiment = better rank

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = lgb.LGBMRegressor()
    model.fit(X_train, y_train)

    df["Recommendation Score"] = model.predict(X)

    return df.sort_values(by="Recommendation Score", ascending=False)



**Rule Engine**

In [39]:
# Match investor profile with stock selection
def match_investor_profile(df, investor_profile):
    df["risk_match"] = 1 - abs(investor_profile["risk_score"] / 10 - df["volatility"])
    df["sector_match"] = df["Sector"].apply(lambda x: 1 if x in investor_profile["preferred_sectors"] else 0)
    df["Final Score"] = (
        df["Sentiment Score"] * 0.4 + df["risk_match"] * 0.3 + df["sector_match"] * 0.3
    )

    # Profile-Based Filtering
    if investor_profile["risk_score"] >= 80:  # High Risk
        df = df[df["volatility"] >= 1.0]
    elif investor_profile["risk_score"] <= 30:  # Low Risk
        df = df[df["volatility"] <= 0.8]

    # if net_worth > 1_000_000:
    #     df = df.sort_values(by="Market Cap", ascending=False)
    # elif net_worth < 100_000:
    #     df = df.sort_values(by="Stock Price", ascending=True)

    # if mortgage > 0 and liquidity < 50_000:
    #     df = df[df["Sharpe Ratio"] >= 1.0]
    return df.sort_values(by="Final Score", ascending=False)


In [40]:
import pandas as pd

def calculate_investor_stock_score_old(investor, stock_df):
    """
    Assigns a score (out of 100) to a stock based on investor profile.
    
    Parameters:
    - investor (dict): Investor profile
    - stock (dict): Stock details
    
    Returns:
    - int: Final score for the stock
    """
    score = 0.0
    risk_score = 0.0
    liquidity = 0.0
    volatility = 0.0
    market_cap = 0.0
    sharpeRatio = 0.0
    p_e_ratio =0.0
    dividend_yield=0.0
    
#     print(stock)
    
    # Risk Score Match (25%) - Scaled from 1 to 100
    if isinstance(investor["risk_score"], (int, float)): 
        risk_score = investor["risk_score"]
    if isinstance(investor["liquidity"], (int, float)): 
        liquidity = investor["liquidity"]          
    if isinstance(stock["volatility"], (int, float)): 
        volatility = stock["volatility"]
    if isinstance(stock["market cap"], (int, float)): 
        market_cap = stock["market cap"]
    if isinstance(stock["p/e ratio"], (int, float)): 
        p_e_ratio = stock["p/e ratio"]
    if isinstance(stock["sharpeRatio"], (int, float)): 
        sharpeRatio = stock["sharpeRatio"]
        
    if isinstance(stock["dividend_yield"], (int, float)): 
        dividend_yield = stock["dividend_yield"]
    
    
    if risk_score <= 20 and volatility < 0.3:
        score += 25
    elif 21 <= risk_score <= 40 and volatility < 0.6:
        score += 25
    elif 41 <= risk_score <= 60 and 0.6 <= volatility <= 1.2:
        score += 25
    elif 61 <= risk_score <= 80 and volatility <= 2.0:
        score += 25
    elif 81 <= risk_score <= 100:
        score += 25  # High-risk investors can take any stock

    # Liquidity Match (20%)
    if liquidity > market_cap * 0.0001:
        score += 20

    if stock["sector"].empty and investor["preferred_sectors"].any():
        # Sector Preference (15%)
        if investor["preferred_sectors"] == stock["sector"]:
            score += 15

    # Market Cap Stability (15%)
    if market_cap > 50_000_000_000:  # Large-cap stock
        score += 15
    elif market_cap > 10_000_000_000:  # Mid-cap stock
        score += 10

    # P/E Ratio (10%) - Favor stocks with a reasonable P/E ratio (10-30)
    if 10 <= p_e_ratio <= 30:
        score += 10

    # Sharpe Ratio (10%) - Higher Sharpe Ratio gets higher points
    if sharpeRatio > 1.0:
        score += 10
    elif sharpeRatio > 0.5:
        score += 5

   # Dividend Yield Stability (5%) - Consistent dividend payers score higher
    if dividend_yield >= 2.0:
        score += 5
    print("score", score)   
    return score

# Example Investor Profile
investor = {
    "net_worth": 250000,
    "liquidity": 50000,
    "risk_score": 3,
    "region": "US",
    "assets": 500000,
    "mortgage": 100000,
    "preferred_sector": "Healthcare"

}

# Example Stock Data
stocks = [
    {"ticker": "AAPL", "sector": "Technology", "market_cap": 2900000000000, "volatility": 1.2, "p_e_ratio": 28, "sharpe_ratio": 1.1, "dividend_yield": 8.51},
    {"ticker": "JNJ", "sector": "Healthcare", "market_cap": 430000000000, "volatility": 0.7, "p_e_ratio": 18, "sharpe_ratio": 0.9, "dividend_yield": 9.83},
    {"ticker": "TSLA", "sector": "Automotive", "market_cap": 850000000000, "volatility": 2.0, "p_e_ratio": 60, "sharpe_ratio": 0.8, "dividend_yield": 5.6},
    {"ticker": "KO", "sector": "Consumer Defensive", "market_cap": 295000000000, "volatility": 0.5, "p_e_ratio": 27, "sharpe_ratio": 0.5, "dividend_yield": 5.51},
]

# Calculate scores for all stocks
# stock_scores = [{"ticker": stock["ticker"], "score": calculate_investor_stock_score(investor, stock)} for stock in stocks]

# # Convert to DataFrame and sort by best score
# df_scores = pd.DataFrame(stock_scores).sort_values(by="score", ascending=False)


In [41]:
import pandas as pd

def calculate_investor_stock_score(investor, stock):
    """Assigns a score (out of 100) to a stock based on investor profile."""
    risk_score = pd.to_numeric(investor.get("risk_score", 0), errors="coerce")
    liquidity = pd.to_numeric(investor.get("liquidity", 0), errors="coerce")
    volatility = pd.to_numeric(stock.get("volatility", 0), errors="coerce")
    market_cap = pd.to_numeric(stock.get("market_cap", 0), errors="coerce")
    p_e_ratio = pd.to_numeric(stock.get("p_e_ratio", 0), errors="coerce")
    sharpe_ratio = pd.to_numeric(stock.get("sharpeRatio", 0), errors="coerce")
    dividend_yield = pd.to_numeric(stock.get("dividend_yield", 0), errors="coerce")

    sector = stock.get("sector", "")
    preferred_sectors = investor.get("preferred_sectors", [])

    score = 0.0

    # Risk Score Match (25%)
    if risk_score <= 20 and volatility < 0.3:
        score += 25
    elif 21 <= risk_score <= 40 and volatility < 0.6:
        score += 25
    elif 41 <= risk_score <= 60 and 0.6 <= volatility <= 1.2:
        score += 25
    elif 61 <= risk_score <= 80 and volatility <= 2.0:
        score += 25
    elif 81 <= risk_score <= 100:
        score += 25

    # Liquidity Match (20%)
    if liquidity > market_cap * 0.0001:
        score += 20

    # Sector Preference (15%)
    if sector and isinstance(preferred_sectors, list) and sector in preferred_sectors:
        score += 15

    # Market Cap Stability (15%)
    if market_cap > 50_000_000_000:
        score += 15
    elif market_cap > 10_000_000_000:
        score += 10

    # P/E Ratio (10%)
    if 10 <= p_e_ratio <= 30:
        score += 10

    # Sharpe Ratio (10%)
    if sharpe_ratio > 1.0:
        score += 10
    elif sharpe_ratio > 0.5:
        score += 5

    # Dividend Yield (5%)
    if dividend_yield >= 2.0:
        score += 5

    return round(score, 2)

# Example Investor Profile
investor = {
    "risk_score": 68.5,
    "liquidity": 50000,
    "preferred_sectors": ["Healthcare", "Technology"]
}

# Example Stock DataFrame
stock_df = pd.DataFrame([
    {"sector": "Healthcare", "volatility": 0.8, "market_cap": 100_000_000_000, "p_e_ratio": 25, "sharpeRatio": 1.2, "dividend_yield": 2.5},
    {"sector": "Technology", "volatility": 1.1, "market_cap": 200_000_000_000, "p_e_ratio": 30, "sharpeRatio": 0.9, "dividend_yield": 1.8},
    {"sector": "Energy", "volatility": 1.5, "market_cap": 50_000_000_000, "p_e_ratio": 15, "sharpeRatio": 1.3, "dividend_yield": 3.0},
])

# Apply function to each row
# stock_df["score"] = stock_df.apply(lambda stock: calculate_investor_stock_score(investor, stock), axis=1)

# print(stock_df[["sector", "score"]])


**Stock Recommneder**

In [42]:


# Recommend Stocks Based on Investor Profile
def recommend_stocks(net_worth, liquidity, risk_score, region, assets, mortgage, preferred_sectors):
    investor_profile= {
          "net_worth": net_worth,
          "liquidity": liquidity,
          "risk_score": risk_score,
          "mortgage": mortgage,
          "region": region,
          "assets": assets,
          "preferred_sectors": preferred_sectors
      }
#   stock_data = [get_stock_data(stock) for stock in stock_list]
    print("stock_list")
    print(stock_list)
    stock_data = collect_stock_data(stock_list)
    print("stock_data")
    print(stock_data)
    ranked_df = train_stock_ranking_model(stock_data)

    stock_data_df = pd.DataFrame(ranked_df).dropna()
#     print("stock_data_df")
#     print(stock_data_df)
#     matched_df = match_investor_profile(stock_data_df, investor_profile)
#     matched_df = calculate_investor_stock_score(investor_profile, stock_data_df)
    
    # Apply function to each row
    stock_data_df["score"] = stock_data_df.apply(lambda stock: calculate_investor_stock_score(investor, stock), axis=1)
    top_5_stocks = stock_data_df.head(5)
    stocks_df = pd.DataFrame(top_5_stocks).dropna()

    return stocks_df


In [43]:
# recommend_stocks(250000,50000, 67.5,"US",500000,100000,"Healthcare")

**AI Explanation**

In [48]:

# AI Justification for Stock Selection
def ai_stock_explanation(stock_info, investor_profile):


    prompt = f"""
    Investor Profile:
    - Net Worth: {investor_profile['net_worth']} USD
    - Liquidity: {investor_profile['liquidity']} USD
    - Risk Score: {investor_profile['risk_score']}
    - Mortgage Debt: {investor_profile['mortgage']} USD
    - Region: {investor_profile['region']}
    - Preferred Sector: {investor_profile['preferred_sectors']}

    Given this investor profile, explain why {stock_info['ticker']} is a suitable stock recommendation.
    Stock Data:
    - Sector: {stock_info['sector']}
    - Market Cap: {stock_info['market cap']}
    - Volatility: {stock_info['volatility']}
    - P/E Ratio: {stock_info['p/e ratio']}
    - Sharpe Ratio: {stock_info['sharpeRatio']}
    - Dividend Yield: {stock_info['dividend_yield']}
    - Sentiment Score: {stock_info['Sentiment Score']}
    - Recommendation Score: {stock_info['Recommendation Score']}

    Provide an easy-to-understand explanation.
    """


    inputs = tokenizer(prompt, return_tensors="pt")
    response = tinyllama_model.generate(**inputs, max_length=500)

    recommendation = tokenizer.decode(response[0], skip_special_tokens=True)
    # Remove Prompt from Output
    clean_output = recommendation.replace(prompt, "").strip()

    print(clean_output)
    return clean_output

In [49]:
investor_profile= {
        "net_worth": 250000,
        "liquidity": 50000,
        "risk_score": 3,
        "mortgage": 100000,
        "region": "US",
        "assets": 150000,
        "preferred_sectors": "Healthcare"
}

best_stocks = recommend_stocks(investor_profile["net_worth"], investor_profile["liquidity"],
                               investor_profile["risk_score"], investor_profile["region"], investor_profile["assets"],
                               investor_profile["mortgage"], investor_profile["preferred_sectors"])
print("best_stocks")
best_stocks
# stock_data = get_stock_data("AAPL")
# stock_data
stock_explanation = ai_stock_explanation(best_stocks, investor_profile)
stock_explanation

stock_list
['SWN', 'ETN', 'ES', 'BBT']
News Response: [{'id': '341a16d3-7adc-3747-aba8-8d3337bca9a0', 'content': {'id': '341a16d3-7adc-3747-aba8-8d3337bca9a0', 'contentType': 'STORY', 'title': 'Is Southwestern Energy Company (SWN) the Best Stock Under $15 To Buy Now?', 'description': '', 'summary': 'We recently compiled a list of 10 Best Stocks Under $15 To Buy Now. In this article, we will look at where Southwestern Energy Company (NYSE:SWN) ranks among the best stocks under $15. While September is usually the toughest month for the stock market, it is delivering solid results this time. Stocks are soaring after […]', 'pubDate': '2024-10-01T14:49:26Z', 'displayTime': '2024-10-01T14:49:26Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/insidermonkey.com/ebee65f2176b900b17d44579449d2d0e', 'originalWidth': 1456, 'originalHeight': 816, 'caption': '', 'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/2AbX4uePn

/Users/vijayalakshmig/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:2208: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])
Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])
News Response: [{'id': 'b32e96a1-ab9a-30db-a4db-228400edf749', 'content': {'id': 'b32e96a1-ab9a-30db-a4db-228400edf749', 'contentType': 'STORY', 'title': 'Eaton (NYSE:ETN) Collaborates On HiZ Wildfire Prevention Tech With 90% Detection Accuracy', 'description': '', 'summary': "Eaton (NYSE:ETN) introduced its new HiZ Protect™ solution aimed at wildfire prevention, enhancing safety in utility distribution systems. This innovation, achieving over 90% accuracy in detecting high-impedan

Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])
Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])
News Response: [{'id': '4039babb-dd12-340f-bb0d-0793f0a0030d', 'content': {'id': '4039babb-dd12-340f-bb0d-0793f0a0030d', 'contentType': 'STORY', 'title': "Is Eversource Energy's (NYSE:ES) ROE Of 5.4% Concerning?", 'description': '', 'summary': 'One of the best investments we can make is in our own knowledge and skill set. With that in mind, this article will...', 'pubDate': '2025-03-13T17:44:57Z', 'displayTime': '2025-03-13T17:44:57Z', 'isHosted': True, 'bypassModal': False, 'previ

Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])
Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  3611,  1476,  4286,    13,    13, 29946, 29889]])
News Response: []
Raw Model Output: tensor([[    1, 11597, 29891,   911,   278, 19688,   313,  9135,  3321, 29892,
         12610,  1230, 29892,  2448,   329,  1705, 29897,   310,   445,  9763,
         29901,   525,  3782,  9763,  3625,   363,   445, 10961,  4286,    13]])
stock_data
  Stock  POSITIVE  NEGATIVE  NEUTRAL ticker       sector    market cap  \
1   ETN         2         0        0    ETN  Industrials  118643408896   
2    ES         2         0        0     ES    Utili

/Users/vijayalakshmig/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:2208: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


The recommendation score of 2.0 indicates that the stock recommendation is suitable for this investor profile. The Sentiment Score of 2 indicates that the stock is considered to be a good investment opportunity. The Recommendation Score of 2.0 indicates that the stock is a suitable recommendation for this investor profile


'The recommendation score of 2.0 indicates that the stock recommendation is suitable for this investor profile. The Sentiment Score of 2 indicates that the stock is considered to be a good investment opportunity. The Recommendation Score of 2.0 indicates that the stock is a suitable recommendation for this investor profile'

In [50]:
import random

# Sample stock data with sentiment scores
stocks = [
    {
        "ticker": "JNJ",
        "sector": "Healthcare",
        "volatility": 0.45,
        "market_cap": 428_000_000_000,
        "p_e_ratio": 18.5,
        "sharpe_ratio": 1.2,
        "dividend_yield": 2.7,
        "sentiment_score": 0.75,  # Placeholder sentiment score (0 to 1)
    },
    {
        "ticker": "AAPL",
        "sector": "Technology",
        "volatility": 1.1,
        "market_cap": 2_500_000_000_000,
        "p_e_ratio": 28.4,
        "sharpe_ratio": 1.5,
        "dividend_yield": 0.6,
        "sentiment_score": 0.85,
    },
    {
        "ticker": "KO",
        "sector": "Consumer Defensive",
        "volatility": 0.6,
        "market_cap": 295_000_000_000,
        "p_e_ratio": 27.9,
        "sharpe_ratio": 0.9,
        "dividend_yield": 3.1,
        "sentiment_score": 0.65,
    },
    {
        "ticker": "MSFT",
        "sector": "Technology",
        "volatility": 1.0,
        "market_cap": 2_700_000_000_000,
        "p_e_ratio": 35.2,
        "sharpe_ratio": 1.7,
        "dividend_yield": 0.8,
        "sentiment_score": 0.9,
    },
    {
        "ticker": "PG",
        "sector": "Consumer Defensive",
        "volatility": 0.55,
        "market_cap": 350_000_000_000,
        "p_e_ratio": 24.3,
        "sharpe_ratio": 1.1,
        "dividend_yield": 2.4,
        "sentiment_score": 0.7,
    },
]

def calculate_recommendation_score(investor, stock):
    """
    Calculates recommendation score for a stock based on investor profile.
    Score is out of 100.
    """
    score = 0

    # Risk Score Match (25%)
    if investor["risk_score"] <= 20 and stock["volatility"] < 0.3:
        score += 25
    elif 21 <= investor["risk_score"] <= 40 and stock["volatility"] < 0.6:
        score += 25
    elif 41 <= investor["risk_score"] <= 60 and 0.6 <= stock["volatility"] <= 1.2:
        score += 25
    elif 61 <= investor["risk_score"] <= 80 and stock["volatility"] <= 2.0:
        score += 25
    elif 81 <= investor["risk_score"] <= 100:
        score += 25  # High-risk investors can take any stock

    # Sector Preference (15%)
    if stock["sector"] in investor["preferred_sectors"]:
        score += 15

    # Market Cap Stability (15%)
    if stock["market_cap"] > 50_000_000_000:
        score += 15

    # P/E Ratio (10%) - Favor stocks with a reasonable P/E ratio (10-30)
    if 10 <= stock["p_e_ratio"] <= 30:
        score += 10

    # Sharpe Ratio (10%) - Higher Sharpe Ratio gets higher points
    if stock["sharpe_ratio"] > 1.0:
        score += 10
    elif stock["sharpe_ratio"] > 0.5:
        score += 5

    # Dividend Yield Stability (5%)
    if stock["dividend_yield"] >= 2.0:
        score += 5

    # Sentiment Score Adjustment (10%)
    score += stock["sentiment_score"] * 10  # Scale sentiment (0-1) to (0-10)

    return round(score, 2)

def recommend_random_stocks(investor_profile):
    """
    Selects three random stocks and provides an explanation with recommendation and sentiment scores.
    """
    recommended_stocks = random.sample(stocks, 3)  # Select three random stocks

    recommendation = f"""**Investor Profile:**
    - Net Worth: {investor_profile['net_worth']} USD
    - Liquidity: {investor_profile['liquidity']} USD
    - Risk Score: {investor_profile['risk_score']}
    - Mortgage Debt: {investor_profile['mortgage']} USD
    - Region: {investor_profile['region']}
    - Preferred Sector: {", ".join(investor_profile['preferred_sectors'])}
    
    Based on the investor's profile, we recommend the following stocks:\n"""

    for stock in recommended_stocks:
        recommendation_score = calculate_recommendation_score(investor_profile, stock)

        recommendation += f"""
    **{stock['ticker']}** ({stock['sector']} Sector):
    - **Volatility**: {stock['volatility']} → Suitable for risk score **{investor_profile['risk_score']}**
    - **Market Cap**: {stock['market_cap']} → Financially strong company
    - **P/E Ratio**: {stock['p_e_ratio']} → {"Fairly valued" if 10 <= stock['p_e_ratio'] <= 30 else "Expensive stock"}
    - **Sharpe Ratio**: {stock['sharpe_ratio']} → {"Good risk-adjusted return" if stock['sharpe_ratio'] > 1 else "Moderate returns"}
    - **Dividend Yield**: {stock['dividend_yield']}% → {"Stable dividend payer" if stock['dividend_yield'] >= 2 else "Low dividend yield"}
    - **Sentiment Score**: {stock['sentiment_score']} (0-1) → {"Positive" if stock['sentiment_score'] > 0.6 else "Neutral/Negative"}
    - **Recommendation Score**: {recommendation_score}/100

    **Why?** This stock aligns with the investor’s profile and provides a balance of growth, stability, and risk.
    """

    return recommendation

# Example investor profile
investor = {
    "net_worth": 500000,
    "liquidity": 100000,
    "risk_score": 55,
    "mortgage": 150000,
    "region": "US",
    "preferred_sectors": ["Technology", "Healthcare"]
}

# Get stock recommendations
print(recommend_random_stocks(investor))


**Investor Profile:**
    - Net Worth: 500000 USD
    - Liquidity: 100000 USD
    - Risk Score: 55
    - Mortgage Debt: 150000 USD
    - Region: US
    - Preferred Sector: Technology, Healthcare
    
    Based on the investor's profile, we recommend the following stocks:

    **PG** (Consumer Defensive Sector):
    - **Volatility**: 0.55 → Suitable for risk score **55**
    - **Market Cap**: 350000000000 → Financially strong company
    - **P/E Ratio**: 24.3 → Fairly valued
    - **Sharpe Ratio**: 1.1 → Good risk-adjusted return
    - **Dividend Yield**: 2.4% → Stable dividend payer
    - **Sentiment Score**: 0.7 (0-1) → Positive
    - **Recommendation Score**: 47.0/100

    **Why?** This stock aligns with the investor’s profile and provides a balance of growth, stability, and risk.
    
    **AAPL** (Technology Sector):
    - **Volatility**: 1.1 → Suitable for risk score **55**
    - **Market Cap**: 2500000000000 → Financially strong company
    - **P/E Ratio**: 28.4 → Fairly valued
 